In [1]:
"""
This script splits our data into peices so that NER can be run in parallel 
"""

'\nThis script splits our data into peices so that NER can be run in parallel \n'

In [10]:
import pandas as pd
import spacy
import numpy as np
from collections import Counter
import csv
from multiprocessing import Pool 

In [11]:
#OLD: inFile = "/shared/3/projects/benlitterer/localNews/mergedNewsData/dataSplits/splitNews27"
inFile = "/shared/3/projects/newsDiffusion/data/processed/newsData/fullMergedNELAdata.tsv"
newsDf = pd.read_csv(inFile, sep="\t")

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (3,4,11,12,13,14,15,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
sub_df = newsDf[['key', 'content']]
len(sub_df)

5081145

In [16]:
## Write the frames to output so that they can be easily analyzed in parallel using a script 

In [17]:
#we use 160 because that's how many cores we have 
indexList = np.array_split(np.array(list(range(0, len(sub_df)))), 160)
zippedIndices = list(zip(list(range(0, 160)), indexList))

outStem = "/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/NERSplits/NERSplits"

#last file will be missing one row 
def writeFrame(tup):
    i, indices = tup
    outDf = sub_df.iloc[indices, :]
    outDf.to_csv(outStem + str(i), sep="\t", quoting=csv.QUOTE_NONNUMERIC)

with Pool(12) as p: 
    p.map(writeFrame, zippedIndices)

